In [ ]:
# Word2Vec
# two models: Skip-Gram and CBow
# CBOW: predict input word based on context
# Skip-Gram: predict context based on input

In [ ]:
# two steps:
# 1. Build model - Fake task - auto encoder
# 2. Get embedding vector from the model

# Ptep 1: Build Model

In [ ]:
# example: the dog barked at the mailman
# input word
# skip_window -> span = skip_window * 2
# num_skips: (input word, output word) * num_skips
# output of the model: probability of words to be collocate

In [ ]:
# example: The quick brown fox jumps over lazy dog
# skip_window = 2

In [ ]:
# how do we represent these words?
# train text -> vocabulary -> one hot encoding
# hyper-parameters: dimensions of word vector; window_size
# hidden layer: each row is the word vector of each word

# hidden layer
# sparse matrix computation
# hidden layer weight matrix -> lookup table
# no need to compute, just look up values in the weight matrix
# where the corresponding input vector is not zero

In [ ]:
# output layer
# softmax classifier, output probability

# example 'ants'
# word vector of ants (1*300, 300 is the num of nodes in hidden layer)
# softmax: (e**x)/(sum(e**x))

# analysis: in case two words share similar context aka similar window words
# Kitty climmed the tree.; Cat climmed the tree.

# stemming: ant - ants

# Part 2: Training Skip_Gram

In [ ]:
# Word2Vec is super big
# Doing Gradient Descent in such a huge NN is time-consuming
# It needs tons of data otherwise it may be overfitted

# word pairs
# sample highly frequent words to reduce the num of training samples
# negative sampling

In [ ]:
# 2.1 Word Pairs and Pharses
# 2.2 Sampling on high-frequency vocabulary
# Sampling rate
# 2.3 Negative Sampling

In [ ]:
# Preprocessing
# Build input for training
# Build model
# Evaluate model

In [1]:
import time
import numpy as np
import tensorflow as tf
import random
from collections import Counter

In [8]:
with open ('/Users/pliu/Downloads/text8.txt', 'r') as f:
    text = f.read()
print(text[:20])

anarchism originated


In [ ]:
# preprocessing
# remove non-char and low-frequency letters
# word-split from text
# construct input
# mapping of vocabulary

In [10]:
def preprocess(text, freq=5):
    """
    process input data
    args:
        text: the text to process
        freq: lower threshold for frequency of a word
    """
    text = text.lower()
    text = text.replace('.','<PERIOD> ')
    text = text.replace(',','<COMMA> ')
    text = text.replace('"','<QUOTATION_MARK> ')
    text = text.replace(';','<SEMICOLON> ')
    text = text.replace('!','<EXCLAMATION_MARK> ')
    text = text.replace('?','<QUESTION_MARK> ')
    text = text.replace('(','<LEFT_PAREN> ')
    text = text.replace(')','<RIGHT_PAREN> ')
    text = text.replace('--','<HYPHENS> ')
    text = text.replace(':','<COLON> ')
    words = text.split()

    # remove low-frequency words to reduce noise
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > freq]
    
    return trimmed_words

In [11]:
print(text[:30])
words = preprocess(text)
print(words[:30])

anarchism originated as a term
['anarchism', 'as', 'a', 'of', 'first', 'the', 'of', 'the', 'and', 'the', 'of', 'the', 'the', 'is', 'in', 'a', 'to', 'that', 'to', 'the', 'of', 'society', 'it', 'as', 'a', 'by', 'anarchists', 'the', 'anarchism', 'is']


In [12]:
# build the mapping 
vocab = set(words)
vocab_to_int = {w:c for c,w in enumerate(vocab)}
int_to_vocab = {c:w for c,w in enumerate(vocab)}

In [13]:
print(len(words))
print(len(set(words)))

549
35


In [15]:
# mapping from vocab to int
int_words = [vocab_to_int[w] for w in words]

In [17]:
int_word_counts = Counter(int_words)

print(int_word_counts)

Counter({15: 78, 21: 55, 7: 40, 3: 37, 32: 24, 16: 20, 23: 20, 27: 19, 31: 18, 6: 15, 33: 14, 19: 13, 10: 13, 12: 13, 25: 13, 34: 10, 18: 10, 2: 9, 13: 9, 24: 9, 0: 9, 26: 9, 14: 8, 20: 8, 4: 8, 5: 8, 30: 8, 8: 7, 17: 7, 11: 7, 29: 7, 1: 6, 9: 6, 22: 6, 28: 6})


In [25]:
# Sampling
t = 1e-3
threshold = 0.8

int_word_counts = Counter(int_words)

# calculate frequency of each word
# total
total_count = len(int_words)
# for each
word_freq = {w: c/total_count for w, c in int_word_counts.items()}
# compute probability to be deleted
prob_drop = {w:1-np.sqrt(t/word_freq[w]) for w in int_word_counts}
# sampling the words
train_words = [w for w in int_words if prob_drop[w] < threshold]

In [27]:
len(train_words)

209

In [28]:
# Batch
# Skip-Gram: predict context by input words
def get_targets(words, idx, window_size):
    """
    get context word list of an input word
    args:
        words: vocab list
        idx: index of input word
        window_size: the size of sliding window
    return:
        list of context words of an input word
    """
    target_window = np.random.randint(1, window_size+1)
    # compute index of start and end point of the sliding window
    # take into account the situation that input word may have no sufficient words in front
    start_point = idx - target_window if (idx - target_window) > 0 else 0
    end_point = idx + target_window
    
    # output words, the context words in window
    targets = words[start_point:idx] + words[idx+1:end_point+1]
    targets = set(targets)
    return list(targets)

In [29]:
def get_batches(words, batch_size, window_size):
    """
    a batch generator
    args:
        words: training words
        batch_size: num of words in each batch
        window_size: sliding window size
    """
    n_batches = len(words) // batch_size
    
    # full batches only
    words = words[:n_batches*batch_size]
    
    # get batch
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx: idx+batch_size]
        
        for i in range(len(batch)):
            batch_x = batch[i]
            batch_y = get_targets(batch, i, word_size)
            
            x.extend([batch_x*len(batch_y)])
            y.extend(batch_y)
            
        yield x, y         

In [ ]:
# Build NN

# input layer
# embedding layer
# negative sampling (accelerating BP)


In [32]:
# input layer
train_graph = tf.Graph()
with train_graph.as_default():
    inputs = tf.placeholder(tf.int32, shape=[None], name='inputs')
    labels = tf.placeholder(tf.int32, shape=[None,None], name='labels')

In [ ]:
# embedding layer
# embedding matrix: vocab_size*hidden_units_size
# embedding matrix as lookup table

In [30]:
vocab_size = len(int_to_vocab)
embedding_size = 200

In [33]:
# construct embedding matrix
with train_graph.as_default():
    # embedding weight matrix
    embedding = tf.Variable(tf.random_uniform([vocab_size, embedding_size], -1, -1))
    # lookup
    embed = tf.nn.embedding_lookup(embedding, inputs)

In [34]:
# Negative Sampling
# to accelerate Gradient Descent
n_sampled = 100
with train_graph.as_default():
    softmax_w = tf.Variable(tf.truncated_normal([vocab_size, embedding_size], stddev=0.1))
    softmax_b = tf.Variable(tf.zeros(vocab_size))

    # calculate loss under negative sampling
    loss = tf.nn.sampled_softmax_loss(softmax_w, softmax_b, labels, embed, n_sampled, vocab_size)
    
    cost = tf.reduce_mean(loss)
    optimizer = tf.train.AdamOptimizer().minimize(cost)

In [36]:
# evaluate
# check similar sentences of inputs
with train_graph.as_default():
    # pick up words randomly
    valid_size = 16
    valid_window = 100
    
    # choose words for valudation from differennt positions
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples, 
                               random.sample(range(1000, 1000+valid_window), valid_size//2))
    
    valid_size = len(valid_examples)
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # standardize each embedding ()
    norm = tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
    normalized_embedding = embedding / norm
    
    # lookup embedding of word for validation
    valid_embedding = tf.nn.embedding_lookup(normalized_embedding, valid_dataset)
    
    # compute cosine similarity
    similarity = tf.matmul(valid_embedding, tf.transpose(normalized_embedding))

In [ ]:
epochs = 10 # umber of iteration
batch_size = 1000 # size of batch
window_size = 10 # size of sliding window

with train_graph.as_default():
    saver = tf.train.Saver()
    
with tf.Session(graph=train_graph) as sess:
    iteration = 1
    loss = 0
    sess.run(tf.global_variables_initializer())
    
    # do iteration for epochs times
    for e in in range(1, epochs+1):
        batches = get_batches(train_words, batch_size, window_size)
        start = time.time()
        
        for x, y in batches:
            feed = {inputs:x,
                    labels:np.array(y)[:,None]}
            train_loss, _ = sess.run([cost, optimizer], feed_dict = feed)
            
            loss += train_loss
            
            # for each 100 times
            if iteration % 100 == 0:
                end = time.time()
                # log training 
                print('Epoch{}/{}'.format(e, epochs),
                      'Iteration: {}'.format(iteration),
                      'Avg Training loss:{:.4f}'.format(loss/100),
                      '{:.4f} sec/batch'.format((end-start)/100))
                
                
        
